In [20]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import spacy
import urllib
import re
import math
import torch
from gensim import corpora
import pickle
import random as rd


In [2]:
tree = ET.parse('arquivos_xml/12002010410.xml')
root = tree.getroot()


In [3]:
for child in root:
    print(child.tag)
    #print(child.root)
    for key in child.attrib:
        print(key)
        print(child.attrib[key])

article
numberPage
6
pubName
DO1
name
PORTARIA
artType
PORTARIA
pubDate
04/01/2002
artCategory
Ministério da Agricultura,/Pecuária e Abastecimento/PROGRAMA DE GARANTIA DA ATIVIDADE AGROPECUÁRIA/COMISSÃO ESPECIAL DE RECURSOS SECRETARIA EXECUTIVA
pdfPage
http://pesquisa.in.gov.br/imprensa/jsp/visualiza/index.jsp?jornal=1&data=04/01/2002&pagina=6
editionNumber
3


In [90]:
filename = 'arquivos_xml/12002010410.xml'
def read_xml(xml_file_location):
    tree = ET.parse(xml_file_location)
    root = tree.getroot()
    soup = None
    docs = ''
    for paragraph in root.iter('Texto'):
        #print(paragraph.text)
        soup = BeautifulSoup(paragraph.text, 'html.parser')
        for p in soup.select('p'):
            texto = p.text.replace('\n', ' ').strip()
            if len(texto) > 0:
                docs = docs + ' ' + texto
        docs = docs.strip().lower()
    return docs

read_xml(filename)

'portaria nº 1, de 2 de janeiro de 2002 o secretário-executivo da secretaria da comissão especial de recursos, no uso de sua competência e das atribuições estabelecidas pelas portarias nº 422, de 8 de outubro de 1997, nº 534, de 10 de novembro de 1998, do ministério da agricultura e do abastecimento, divulga, na forma do disposto no art. 3º da resolução nº 2.427, de 1º de outubro de 1997, do banco central do brasil, informações para efeito do zoneamento agrícola, contemplando épocas indicadas para implantação da cultura/custeio de entressafra, cultivares, tipos e aptidão dos solos. art. 1º - cultura temporária: algodão herbáceo. art. 2º - cultura permanente: algodão arbóreo. art. 3º - safra 2002/2003 - estado do rio grande do norte. art. 4º - as pertinentes discriminações estão contidas em anexo, observado o disposto nas resoluções nos  2.403 e 2.422, de 25 de junho e 10 de setembro de 1997, respectivamente, do banco central do brasil. art. 5º- esta portaria entra em vigor na data de s

In [6]:
nlp = spacy.load("pt_core_news_sm")


In [81]:
def tokenizer(text, spacy_model=None, lemm_alternatives=None, exeption_lst=['ADJ', 'PROPN', 'NOUN'], insert_val_mask=True, debug=False):
    last_viewed = None
    text_split = re.split(r'(\W+)', text)
    #text_split = re.sub(r'\W+', ' ', text)
    if spacy_model == None:
        nlp = spacy.load("pt_core_news_sm")
    else:
        nlp = spacy_model
        
    model = nlp(' '.join(text_split).strip())
    #model = nlp(text_split.strip())
    tokens = []
    new_tokens = []
    count = 0
    for word in model:
        str_word = str(word.text).lower().strip()
        if str_word != last_viewed:
            if count > 0:
                tokens += new_tokens + ['##SEQT']
            elif last_viewed != None:
                tokens += new_tokens
                
            new_tokens = []
            last_viewed = str_word
            count = 0
            
            num = True
            len_word = 0
            
            if lemm_alternatives and str_word in lemm_alt:
                    str_lemma = lemm_alternatives[str_word].lower()
            else:
                str_lemma = str(word.lemma_).lower().strip()
                
            try:
                int_word = int(str_word)
            except:
                num = False

            if num:
                new_tokens.append('##NUM')
                if insert_val_mask:
                    new_tokens.append(str_word)
                if debug:
                    print(str_word)
                    print(f'num: {word.pos_}')
            elif word.pos_ == 'NUM':
                new_tokens.append('##NUMEXT')
                if insert_val_mask:
                    new_tokens.append(str_word)
                if debug:
                    print(str_word)
                    print(f'num: {word.pos_}')
            elif word.pos_ in exeption_lst:
                if debug:
                    print(str_word)
                    print(f'in exeption list: {word.pos_}')
                tokens.append(str_word)
            elif len(str_word) > 0 and str_word == str_lemma:
                new_tokens.append(str_word)
            else:
                #lemma
                i = 0
                
                if len(str_lemma) < len(str_word):
                    len_word = len(str_lemma)
                else:
                    len_word = len(str_word)

                while i < len_word and str_word[i] == str_lemma[i]:
                    i += 1
                #print(str_word, str_lemma)
                if i == 0 and len(str_word) > 0:
                    new_tokens.append(str_word)              
                    new_tokens.append('##' + str_lemma)
                elif len(str_word[:i].strip()) > 0:
                    new_tokens.append(str_word[:i].strip())

                if i != 0 and len(str_word[i:].strip()) > 0:
                    new_tokens.append('##' + str_word[i:].strip())

                if debug:
                    print(i)
                    print(str_word)
                    print(str_lemma[0:i], str_lemma[i:])
                    print(word.pos_)
        else:
            count += 1
                
    return tokens + new_tokens
    
def pack_tokens(tokens, dictionary, max_length= 512):
    nr_sep = math.ceil(len(tokens)/max_length)
    max_length_sep = max_length - nr_sep
    blks = math.ceil(len(tokens)/max_length_sep)
    inputs = []
    blocks = []
    masks = []
    dictionary.add_documents([tokens])
    for i in range(blks):
        mask = [1] * max_length
        offset = max_length_sep*i
        #print(len(tokens[offset:(max_length_sep + offset)]))
        res = ["[CLS]"] + tokens[offset:(max_length_sep + offset)] + ["[SEP]"]
        if len(res) < max_length:
            for i in range(len(res), max_length):
                mask[i] = 0
                res.append('[PAD]')
        #print(len(res))
        inputs.append(mask_block(res, dictionary))
        blocks.append(res)
        masks.append(mask)
    return (inputs, blocks, masks)

def mask_block(block, dictionary=None):
    prob = rd.random()
    if prob <= 0.1:
        return block
    else:
        mask_pos = rd.randint(1, block.index("[SEP]"))
        print(mask_pos)
        if block[mask_pos] not in ["[CLS]", "[SEP]", "[PAD]"]:
            if prob <= 0.2:
                if dictionary == None:
                    mask_pos2 = rd.randint(1, block.index("[SEP]")-1)
                    if mask_pos2 != mask_pos:
                        return block[0:mask_pos] + block[mask_pos2] + block[mask_pos+1:]
                    else:
                        return block[0:mask_pos] + block[mask_pos2+1] + block[mask_pos+1:]
                else:
                    mask_pos2 = rd.randint(4, len(dictionary)-1)
                    if dictionary.token2id[block[mask_pos2]] != mask_pos:
                        return block[0:mask_pos] + [dictionary[mask_pos2]] + block[mask_pos+1:]
                    else:
                        return block[0:mask_pos] + [dictionary[mask_pos2+1]] + block[mask_pos+1:]
            else:
                return block[0:mask_pos] + ['[MASK]'] + block[mask_pos+1:]
    return block

In [34]:
ms[310]

'[MASK]'

In [35]:
packs[0][0][310]

'cambissolos'

In [33]:
ms = mask_block(packs[0][0])


310


In [82]:
dictionary = corpora.Dictionary()
dictionary.add_documents([["[CLS]"], ["[SEP]"], ["[MASK]"], ["[PAD]"]])
packs = pack_tokens(tokenizer(docs, insert_val_mask=False), dictionary)
print(len(packs))
print(len(packs[0][0]))
print(len(packs[0][1]))

118
3
512
512


In [80]:
dictionary[0]

'[CLS]'

In [87]:
packs[1][0][118]

'entressafra'

In [ ]:
dataset = []
dictionary = corpora.Dictionary()
dictionary.add_documents([["[CLS]"], ["[SEP]"], ["[MASK]"], ["[PAD]"]])

for texto in docs:
    #print("Processing:", len(texto))
    tk = tokenizer(el, spacy_model=nlp, lemm_alternatives=lemm_alt, insert_val_mask=False, debug=False)
    res, ms = pack_tokens(tk)
    dictionary.add_documents(res)
    for i, r in enumerate(res):
        dataset.append({"doc": r, "mask": ms[i]})
            
for item in dataset:
    elems = []
    for el in item["doc"]:
        elems.append(dictionary.token2id[el])
    item["encoded"] = elems
    

In [93]:
dictionary.token2id[dictionary[len(dictionary)-1]]

242